In [1]:
from qiskit import (Aer, ClassicalRegister, QuantumCircuit,
                    QuantumRegister, execute)


In [53]:
import random

In [69]:
def controller(fzc, mycircuit, qreg, phrases,ancilla,case):
    #qubit_list = phrases.append(qreg)
    if case == 0:
        mycircuit.append(fzc, [3,0,1,2,4] )
    if case == 1:
        mycircuit.x(3)
        mycircuit.append(fzc, [3,0,1,2,4] )
        
        
    r = random.choice(variables)
    mycircuit.x(ancilla[0])
    mycircuit.cx(ancilla[0], qreg[r])
    mycircuit.barrier()
    mycircuit.reset(ancilla[0])

In [70]:
def clause_variables(clause):
    return [abs(int(c))-1 for c in clause]


In [71]:
def findZeroClauseN(mycircuit, variables, ancilla, clause):
    clauseToschoning = clause_variables(clause)
    for c in clause:
        if int(c) < 0:
            mycircuit.x(variables[abs(int(c))-1])

    for i in clauseToschoning:
        mycircuit.x(i)
    print(clause,clauseToschoning)

    mycircuit.mcx(variables[clauseToschoning], ancilla[0])
    mycircuit.x(ancilla[0])

    for i in clauseToschoning:
        mycircuit.x(variables[i])

    for c in clause:
        if int(c) < 0:
            mycircuit.x(variables[abs(int(c))-1])

    return mycircuit

In [72]:
from math import ceil, log

line = [
    [-1, 2, 3],
    [-1, -2, -3]]
no_qubits = 3
no_phrases = 2

variables = QuantumRegister(no_qubits)
phrases = QuantumRegister(math.ceil(log(no_phrases)))
print(len(phrases))
ancilla = QuantumRegister(1)
creg = ClassicalRegister(no_qubits)
qc = QuantumCircuit(variables, phrases, ancilla, creg)

qc.h(variables)
qc.h(phrases)

fzc_array = []
for l in line:
    fzc_array.append(findZeroClauseN(QuantumCircuit(variables,ancilla), variables, ancilla, l).to_gate().control(1))
    


#case 0

variables = clause_variables(line[0])
controller(fzc_array[0], qc, variables, phrases,ancilla,0)

#case 1
variables = clause_variables(line[1])
controller(fzc_array[1], qc, variables, phrases,ancilla,1)



tuple_list = []

qc.measure(variables , creg)
job = execute(qc, Aer.get_backend("qasm_simulator"), shots=1000)
counts = job.result().get_counts(qc)


1
[-1, 2, 3] [0, 1, 2]
[-1, -2, -3] [0, 1, 2]


In [73]:
print(counts)

{'100': 211, '000': 116, '010': 134, '101': 56, '001': 105, '011': 180, '111': 67, '110': 131}


In [74]:
qc.draw()

┌───┐┌───────────────┐     ┌───┐ ░      ┌───────────────┐           ░ »
q250_0: ┤ H ├┤0              ├─────┤ X ├─░──────┤0              ├───────────░─»
        ├───┤│               │     └─┬─┘ ░      │               │           ░ »
q250_1: ┤ H ├┤1              ├───────┼───░──────┤1              ├───────────░─»
        ├───┤│               │       │   ░      │               │     ┌───┐ ░ »
q250_2: ┤ H ├┤2 circuit-2042 ├───────┼───░──────┤2 circuit-2080 ├─────┤ X ├─░─»
        ├───┤│               │       │   ░ ┌───┐│               │     └─┬─┘ ░ »
q251_0: ┤ H ├■               ├───────┼───░─┤ X ├■               ├───────┼───░─»
        └───┘│               │┌───┐  │   ░ └───┘│               │┌───┐  │   ░ »
q252_0: ─────┤3              ├┤ X ├──■───░──|0>─┤3              ├┤ X ├──■───░─»
             └───────────────┘└───┘      ░      └───────────────┘└───┘      ░ »
 c25: 3/══════════════════════════════════════════════════════════════════════»
                                                                              »
«             ┌─┐      
«q250_0: ─────┤M├──────
«             └╥┘┌─┐   
«q250_1: ──────╫─┤M├───
«              ║ └╥┘┌─┐
«q250_2: ──────╫──╫─┤M├
«              ║  ║ └╥┘
«q251_0: ──────╫──╫──╫─
«              ║  ║  ║ 
«q252_0: ─|0>──╫──╫──╫─
«              ║  ║  ║ 
« c25: 3/══════╩══╩══╩═
«              0  1  2

In [75]:
from qiskit import transpile
basis_gates = ['h', 'cx', 'x']
transpiled = transpile(qc, basis_gates=basis_gates)
transpiled.draw()

TranspilerError: "Unable to map source basis {('barrier', 5), ('measure', 1), ('ccx', 3), ('cp', 2), ('reset', 1), ('x', 1), ('cu', 2), ('h', 1), ('cx', 2)} to target basis {'delay', 'cx', 'barrier', 'measure', 'snapshot', 'h', 'x', 'reset'} over library <qiskit.circuit.equivalence.EquivalenceLibrary object at 0x0000019DBEEAFD30>."

In [ ]:
transpiled.draw()